In [ ]:
import sys, os.path, os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    path_opencp="drive/Shareddrives/FAIRNESS/Colab/Librerias/PredictCode/"
    path_fairness="drive/Shareddrives/FAIRNESS/Colab/Librerias/"
    path_exp="drive/Shareddrives/FAIRNESS/Colab/Notebooks_for_topic/EXP0/SCRIPTS/"

except:

    path_opencp="g:\\Unidades compartidas\\FAIRNESS\\Colab\\Librerias\\PredictCode"
    path_fairness="g:\\Unidades compartidas\\FAIRNESS\\Colab\\Librerias"
    path_exp="g:\\Unidades compartidas\\FAIRNESS\\Colab\\Notebooks_for_topic\\EXP0\\SCRIPTS"

sys.path.insert(0, os.path.abspath(path_opencp))
sys.path.insert(0, os.path.abspath(path_fairness))
sys.path.insert(0, os.path.abspath(path_exp))

In [ ]:
import open_cp
import numpy as np
import open_cp.plot as plot
import matplotlib.pyplot as plt
import matplotlib
import datetime
import pickle as pkl

try:
    os.chdir("drive/Shareddrives/FAIRNESS/Colab/Notebooks_for_topic/EXP0/")
except:
    os.chdir("g:\\Unidades compartidas\\FAIRNESS\\Colab\\Notebooks_for_topic\\EXP0")

Failed to load 'descartes' package.


In [ ]:
def plot_events(data,f_inicial,f_final):
    #Data ¡no manipulada!
    fig, ax = plt.subplots(ncols=2, figsize=(9,2.5))

    ax[0].scatter(data.xcoords, data.ycoords, marker="+", alpha=0.5)
    ax[0].set_title("Coordinates")
    ax[0].set_xlabel('Coordinate X')
    ax[0].set_ylabel('Coordinate Y')
    ax[0].set(xlim=[0,1], ylim=[0,1])

    times = data.times_datetime()
    ax[1].scatter(times, data.ycoords, marker="+", alpha=0.1)
    ax[1].set_xlim([f_inicial, f_final])
    ax[1].set_ylim([0, 1])
    ax[1].set_title("Coordinate Vs Dates")
    ax[1].set_xlabel('Date')
    ax[1].set_ylabel('Coordinate Y')
    fig.autofmt_xdate()
    None

def plot_intensity(region,grid_pred,data):
    fig, ax = plt.subplots(figsize=(9,2.5))
    ax.set(xlim=[region.xmin, region.xmax], ylim=[region.ymin, region.ymax])
    m = ax.pcolormesh(*gridpred.mesh_data(), gridpred.intensity_matrix.data, cmap="jet",vmin=0)
    fig.colorbar(m, ax=ax)
    ax.scatter(data.xcoords, data.ycoords, alpha=0.1, color='black')
    ax.set_xlabel('Coordinate X')
    ax.set_ylabel('Coordinate Y')

##<center> <font color='green'> **0. Setup variables**

In [ ]:
from open_cp.sources.sepp import make_time_unit
from global_vars import x_min,x_max,y_min,y_max, f_inicial,f_final,grid_size,days_time_unit,n_events_day


### region de estudio
region = open_cp.RectangularRegion(x_min,x_max, y_min,y_max)
### definicion unidad de tiempo (delta_t)
time_unit = make_time_unit(datetime.timedelta(days=days_time_unit))

total_days=(f_final-f_inicial).days ## dias de estudio totales
number_of_samples=30 ## cantidad de conjuntos de datos


### sampler of kernel spatial given region
from open_cp.sources.random import KernelSampler


### time kernels
from open_cp.sources.sepp import Exponential,HomogeneousPoisson


### kernels espacio-temporales producto entre los kernels
from open_cp.sources.sepp import InhomogeneousPoissonFactors


### para calcular la frecuencia datos reales en grilla

import open_cp.naive as naive

##<center> <font color='green'> **1. funcion para generar datos SEPP dados los kernels**

In [ ]:
#### kernel spacial, conformado por hotspots y sus varianzas
from open_cp.kernels import GaussianKernel

def sepp_sample(means_hotspots, variances_hotspots,total_rate_exp,variance_trigg,rate_time_kernel,f_inicial,time_unit,region,**args):
    ### definicion de kernel gaussiano con varios hotspots
    GK=GaussianKernel(means_hotspots.T,variances_hotspots.T)
    try:
        GK.set_scale(1/max(GK(means_hotspots.T)))
    except:
        GK.set_scale(1/GK(means_hotspots.T))
    K_sampler=KernelSampler(region,GK,1) ## kernel sampler
    ### definicion de kernel temporal back
    Homo_timeK=HomogeneousPoisson(
        rate=rate_time_kernel # the expected number of events per time unit.
    )

    Exp_timeK=Exponential(1, ## The "rate" parameter of the exponential.
                      total_rate_exp ## expected number of events.
                      )

    background_sampler = open_cp.sources.sepp.InhomogeneousPoissonFactors(Homo_timeK, K_sampler)

    trig_space_sampler = open_cp.sources.sepp.GaussianSpaceSampler([0,0],[variance_trigg,variance_trigg],0)

    trigger_sampler = open_cp.sources.sepp.InhomogeneousPoissonFactors(
        Exp_timeK, trig_space_sampler)

    simulation = open_cp.sources.sepp.SelfExcitingPointProcess(background_sampler, trigger_sampler)
    sample = simulation.sample(0, total_days)

    timed_points = open_cp.sources.sepp.scale_to_real_time(sample, f_inicial, time_unit)

    return timed_points


##<center> <font color='green'> **2. funcion para generar variables aleatorias**

In [ ]:
## genera aleatoriamente las variables necesarias para simular los eventos con SEPP

def random_vars():

    ## cantidad de hotspots espaciales
    n_spatial_hotspots=np.random.randint(1,5)
    ## means of hotspots
    centros_hotspots=np.random.rand(n_spatial_hotspots,2)
    varianzas=np.random.rand(n_spatial_hotspots,2)*0.04
    exp_rate=np.random.rand()*0.05+0.7
    trigger_var=np.random.rand()*0.001

    return {"means_hotspots":centros_hotspots,
            "variances_hotspots":varianzas,
            "total_rate_exp":exp_rate,
            "variance_trigg":trigger_var,
            "rate_time_kernel":n_events_day/3,
            "n_spatial_hotspots": n_spatial_hotspots
            }

##<center> <font color='green'> **3. simulacion y almacenamiento**

In [ ]:
from global_vars import dir_sims
dir_parametros=os.path.join(dir_sims,"parametros")
prefix_data="Data_"

In [ ]:
for i in range(number_of_samples):
    parms=random_vars()
    parms["f_inicial"]=f_inicial
    parms["time_unit"]=time_unit
    parms["region"]=region

    timed_points=sepp_sample(**parms)
    print(timed_points.number_data_points)
    plot_events(timed_points,f_inicial,f_final)
    #pkl.dump(parms, open(os.path.join(dir_parametros,prefix_data+str(i)), "wb"))
    pkl.dump(timed_points, open(os.path.join(dir_sims,prefix_data+str(i)+".pkl"), "wb"))

Output hidden; open in https://colab.research.google.com to view.